## Purpose
The purpose of this notebook to load in all the images and their associated labels, then make a new image which is cropped based on the labels (essentially giving us our region of interest). Those images will be used to train the K-Means clustering, and also will be used for generating synthetic "unhealthy" images.

The following cell imports the needed libraries

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image

The following cell creates the directory where all the png images will be saved.

In [2]:
try:
    if not os.path.exists('cropped_png_images'):
        os.makedirs('cropped_png_images')
except OSError:
    print('Error creating data directory.')

In [3]:
import os
from PIL import Image

def crop_and_resize(image_path, label_path):
    """Crops a 256x256 image to the region of interest (ROI) specified by the label file and resizes it back to 256x256."""
    # Verify that the image and label file have matching base names.
    image_base = os.path.splitext(os.path.basename(image_path))[0]
    label_base = os.path.splitext(os.path.basename(label_path))[0]
    if image_base != label_base:
        raise ValueError("Image file and label file names do not match.")

    # Open the image.
    image = Image.open(image_path)
    if image.size != (256, 256):
        raise ValueError("The input image is not 256x256 in size.")

    # Read and parse the label file.
    with open(label_path, 'r') as f:
        line = f.readline().strip()
        parts = line.split()
        if len(parts) != 5:
            raise ValueError("Label file does not contain exactly 5 values.")
        # Parse values: ignoring the first value which represents the class label.
        try:
            _, x_center, y_center, box_width, box_height = parts
            x_center = float(x_center)
            y_center = float(y_center)
            box_width = float(box_width)
            box_height = float(box_height)
        except ValueError:
            raise ValueError("One or more of the coordinate values are not valid floats.")

    # Calculate pixel coordinates for cropping.
    img_width, img_height = image.size
    left   = int((x_center - box_width / 2) * img_width)
    top    = int((y_center - box_height / 2) * img_height)
    right  = int((x_center + box_width / 2) * img_width)
    bottom = int((y_center + box_height / 2) * img_height)
    
    # Ensure coordinates are within image boundaries.
    left = max(0, left)
    top = max(0, top)
    right = min(img_width, right)
    bottom = min(img_height, bottom)

    # Crop the image to the region of interest.
    cropped_image = image.crop((left, top, right, bottom))

    # Resize cropped image back to 256x256 using a high-quality filter.
    resized_image = cropped_image.resize((256, 256), Image.Resampling.LANCZOS)

    return resized_image

In [4]:
def crop_all_images(image_input_dir, label_input_dir, output_dir): 
    """Crops all images to the region of interest (ROI) based on labels, resizes them, and saves them as PNG in output_dir."""
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for filename in os.listdir(image_input_dir):
        img_path = os.path.join(image_input_dir, filename)

        # Check if the image is a PNG file
        if filename.lower().endswith(".png"):
            label_filename = os.path.splitext(filename)[0] + ".txt"
            label_path = os.path.join(label_input_dir, label_filename)

            # Check if the associated label file exists
            if os.path.exists(label_path):
                try:
                    # Crop and resize the image
                    result_image = crop_and_resize(img_path, label_path)
                    
                    # Save the processed image in the output directory
                    result_image.save(os.path.join(output_dir, filename), format="PNG")
                except Exception as e:
                    print(f"Error processing {filename}: {e}")
            else:
                print(f"Skipping {filename}: Label file {label_filename} not found.")


In [6]:
crop_all_images("/data/ai_club/team_13_2024-25/VIPR/Data/YOLO_images_total/", "/data/ai_club/team_13_2024-25/VIPR/Data/YOLO_labels_total/", "/data/ai_club/team_13_2024-25/VIPR/Data/ROI_cropped_images")